In [1]:
import argparse
import torch
import pandas as pd
import json
import numpy as np
from dataset import NorecDataset
from transformers import AdamW, BertForSequenceClassification
import tqdm
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from sklearn.metrics import classification_report
import shap
import scipy as sp

In [2]:
test_sentence = "Fullstendig svikt i klimaforståelsen. Tidligere klima- og miljøminister Ola Elvestuen (V) raser etter at Ap, Sp og Frp skrotet dagens forbud mot nydyrking av myr. Han mener dette viser hva man kan vente seg for norsk klimapolitikk dersom det skulle bli regjeringsskifte. Fredag vedtok et stortingsflertall å oppheve forskriften som forbyr nydyrking av myr i Norge. Det var Ap, Sp og Frp som til slutt kom til enighet om å oppheve forskriften som ble innført som et klimatiltak så seint som i fjor. Disse tre partiene er de mest klima- og naturfiendtlige partiene på Stortinget, raser tidligere klima- og miljøminister Ola Elvestuen (V) til Dagbladet. Han mener Aps beslutning dreier seg om ikke å tape velgere til Sp. Det var Elevstuen som var klima- og miljøminister da forbudet ble vedtatt i 2019. Årsaken var at myr slipper ut enorme mengder CO₂ og andre klimagasser, når den grøftes og graves opp. Likevel kom ikke forskriften på plass før omkring ett år seinere, i fjor sommer. Det var Sp som i utgangspunktet sto bak den nå vedtatte helomvending om å oppheve forbudet mot nydyrking av myr, men Ap kom på banen med sitt eget forslag. Hovedforskjellen fra det opprinnelige forslaget, er at det sidestiller nydyrking og nedbygging, skriver Nationen."

In [3]:
test_sentence2 = """
Bruk-og-kast-mentaliteten var i søkelyset da NRK utfordret norske kjendiser og deres forbruksvaner i programserien Sløsesjokket i vinter. Ved å studere vaner og mønstre, avdekket serien at potensialet for bedre utnyttelse av ressurser var enormt – både i Forsvaret og i offentlig sektor.

Når vi snakker om bærekraft, er det ofte hodeløs bruk av materielle ressurser eller naturressurser som står sentralt. Sløsing med menneskelige ressurser derimot, hører vi ikke like mye om, – og dette til tross for at hele 8 av de 17 bærekraftmålene til FN er såkalte «sosiale» mål.

Mens klima, miljø og grønn omstilling er innarbeidet som naturlige assosiasjoner med begrepet bærekraft, får det sosiale fotavtrykket og menneskets plass i bærekraft-debatten mindre oppmerksomhet. Dette mener jeg tiden er moden for å endre på.

Arbeidsledigheten
Som følge av koronapandemien er 100.000 flere nordmenn blitt arbeidsledige. Unge under 30 år er spesielt hardt rammet, også lokalt: Tall fra Nav viser at antallet unge uten arbeid i aldersgruppa 16 til 29 år i Rogaland nær har doblet seg siden februar 2020.

Selv om de dystre tallene forhåpentligvis er midlertidige, er det grunn til uro over de mange unge som blir stående utenfor arbeidslivet.

Innen industrien er det særlig to grupper jeg bekymrer meg for: de unge permitterte og lærlingene. Det er samfunnsmessig lite bærekraftig at debutantene i arbeidslivet opplever å være overflødige. Sløsingen med de ressursene som våre yngste arbeidstakere er og har, kan komme til å true vår fremtidige velstand på sikt.

Mennesket som ressurs
Sirkulær økonomi handler om å forlenge den tiden en ressurs kan være til nytte i en verdikjede eller et kretsløp. Ettersom vi vet at forventet tid i arbeidslivet vil øke med befolkningens levealder, er det viktig å i større grad ta vare på mennesket som ressurs i den sirkulære tankegangen. Jeg mener at Stavanger som aspirerende energihovedstad og alle vi som er avhengige av at kompetent arbeidskraft er tilgjengelig i regionen, har et særskilt ansvar for å tilby muligheter til de som ufrivillig står utenfor arbeidslivet.

Som daglig leder i en lokal bedrift som leverer personell til olje og gass- og fornybar-industrien, ser jeg med uro på at unge – men også eldre arbeidstakere – taper konkurransen om jobbene. Jeg tror at motivasjonen og fremtidstroa synker for hvert eneste avslag de får. De samme menneskene blir oppmuntret til å se lyst på endring og omstilling i lys av skiftet fra en «sort» til en grønnere økonomi.

Vi har et felles ansvar for å skape en positiv nysgjerrighet rundt det grønne skiftet. Gjennom bærekraftmål 8, forplikter vi oss til å legge til rette for «anstendig arbeid og økonomisk vekst». Skal vi lykkes med dette, mener jeg at vi må sette menneskene i førersetet av prosessen og legge til rette for en mer helhetlig bærekrafttankegang der den sosiale biten er fullt ut inkludert.

Kontinuerlig utdanning
Endring og utvikling krever omstilling og handling i bedriftene. Hvordan kan næringslivet samarbeide for å finne løsninger som sikrer langsiktig bygging av kompetanse for det brede lag av arbeidstakere?

Én måte å gjøre det på, er gjennom kontinuerlig utdanning av arbeidstakere og arbeidssøkere. Vi i selskapet Abelon ønsker å bidra til slik omstilling og inkludering gjennom fremtidsrettet kompetanseheving. Takket være midler fra Rogaland fylkeskommune, utvikler Abelon nå egne kurs innen fagområdet preservering i prosjektet «Omstilling nytt marked og kompetansetilførsel til Rogaland». Med andre ord finnes det støtte fra det offentlige om idéene kommer!

Jeg oppfordrer alle til å tenke på det menneskelige aspektet i bærekraft. I vårt tilfelle driver vi opplæring i preservering, som gjør at teknisk utstyr varer lenger. En viktig ringvirkning er at våre kunder lettere kan etterleve og nå egne bærekraftmål. Men det aller viktigste er kanskje at denne typen kompetansehevende tiltak bidrar til å preservere, eller bevare, kompetansen som regionen trenger for å befeste sin posisjon som fremtidens energihovedstad.

Bærekraftmål nummer 17 handler om å samarbeide for å nå målene, og jeg oppfordrer derfor til et tettere samarbeid for å sikre inkludering og muligheter for alle arbeidstakere som opplever utenforskap. Sammen kan vi unngå at det menneskelige sløsesjokket blir en realitet.

Hva kan din virksomhet bidra med?"""

In [8]:
t3 = """en som måtte betvile Anthony Hopkins sin legitimitet som Oscar-vinner over forhåndsfavoritten Chadwick Boseman, får alle tenkelige bevis på hans briljans i «The Father». Hovedrollen er et utstillingsvindu for den walisiske veteranens spennvidde, fordi den krever alt fra elskverdig sjarme og avvæpnende humor til bitende sarkasme og nærmest psykopatisk sinne.

Han er perfekt til å lede en rørende historie om de praktiske og emosjonelle utfordringene ved fremskreden demens, med like strålende Olivia Colman som utsøkt motspiller."""

In [9]:
def evaluate(device, test_dataset, num_labels, tokenizer, path_to_model_folder, batch_size):
    model = BertForSequenceClassification.from_pretrained(path_to_model_folder)
    model = model.to(device)
    model.eval()
    total_test_acc = 0.
    passes = 0

    def f(x):
        tv = torch.tensor([tokenizer.encode(
            v, padding='max_length', max_length=500, truncation=True) for v in x])
        outputs = model(tv)[0].detach().cpu().numpy()
        scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
        val = sp.special.logit(scores[:, 1])  # use one vs rest logit units
        return val

    explainer = shap.Explainer(f, tokenizer)
    
    docs = test_dataset.documents
    docs.append(test_sentence)
    
    shap_values = explainer(["Denne modellen gjør det sterkt innenfor kulturanmeldelser, da det er dette den er finjustert på. Når det kommer til andre temaer, derimot, presterer den ikke riktig godt. På evalueringen oppnådde modellen en god poengsum. I praksis gjør den det ganske dårlig.", t3])
    shap.plots.text(shap_values)

In [10]:
device = torch.device("cuda" if False else "cpu")
model_data = torch.load("saga_models/models/test/test.pt", map_location=device)
tokenizer = BertTokenizer.from_pretrained("NbAiLab/nb-bert-base")
name = "Test"
print(f"Evaluating model: {name}.pt")

test_dataset = NorecDataset(
    "./data/test.csv")
test_loader = DataLoader(
    test_dataset, batch_size=8, shuffle=True)
num_labels = 2

evaluate(device, test_dataset, num_labels,
     tokenizer, "saga_models/models/test/", 8)

Evaluating model: Test.pt


  0%|          | 0/48 [00:00<?, ?it/s]

Partition explainer:  50%|█████     | 1/2 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

Partition explainer: 3it [02:25, 72.69s/it]               
